This is from this [SO post](https://stackoverflow.com/a/1594484)

### Decorator Basics

#### Python's functions are objects.

To understand decorators, you must first understand that functions are objects in Python. This has important consequences. Let's see why with a simple example:

In [1]:
def shout(word = "yes"):
    return word.capitalize()+"!"

print(shout())

Yes!


In [2]:
#As an object you can assign the function to a variable like any other object
scream =  shout
#Notice we don't use parentheses: we aren't calling the function,
#we're putting the function the function 'shout' into the variable 'scream'
#it means you can then call 'shout' from 'scream'.

print(scream())

Yes!


In [3]:
#More than that, it means you can remove the old name 'shout' and the function will still
#be accessed from 'scream'

del shout
try:
    print(shout())
except NameError as e:
    print(e)

name 'shout' is not defined


In [4]:
print(scream())

Yes!


Keep in mind, we'll circle back to it shortly. 

Another interesting property of Python functions is they can be defined inside another function!

In [5]:
def talk():
    #you can define a function on the fly in 'talk'
    def whisper(word = 'yes'):
        return word.lower()+"..."
    #...and use it right away!
    print(whisper())
    
#you call 'talk', that defines 'whisper' Every time you call it, then 'whisper' is called in 'talk' 
talk()

yes...


In [6]:
#But 'whisper' doesn't outside of 'talk'

try:
    print(whisper())
except NameError as e:
    print(e)

name 'whisper' is not defined


#### Functions references

Okay, still here? Now the fun part...

You've seen that functions are objects. Therefore functions: 

 - can be assigned to a variable
 - can be defined in another function
 
That means that **a function can `return` another function.** 

In [7]:
def getTalk(kind = 'shout'):
    #we define functions on the fly
    def shout(word = 'yes'):
        return word.capitalize()+'!'
    
    def whisper(word = 'yes'):
        return word.lower()+'....'
    
    #Then we return one of them
    if kind == 'shout':
        #we don't use '()' here, we aren't calling the function
        #we are returning the function object
        return shout
    else:
        return whisper

#How do you use this strange beast?
#Get the function and assign it to a variable

talk = getTalk()

#you can see that 'talk' is here a function object

print(talk)
    
    

<function getTalk.<locals>.shout at 0x033E38A0>


In [8]:
print(talk())

Yes!


In [9]:
#And you can even use it directly if you feel wild

print(getTalk("whisper")())

yes....


There's more!

If you can `return` a function, you can pass one as a parameter:

In [10]:
def doSomethingBefore(func):
    print("I do something before then I call the function you gave me")
    print(func())
    
doSomethingBefore(scream)    

I do something before then I call the function you gave me
Yes!


Well, you just have everything needed to understand decorators. You see, decorators are 'wrapers', which means that **they let you execute code before and after the function they decorate** without modifying the function itself.

#### Handcrafted decorators

How you'd do it manually:


In [11]:
#A decorator is a function that expects Another function as parameter
def my_new_decorator(a_function_to_decorate):
    
    #Inside, the decorator defines a function on the fly: the wrapper.
    #This function is going to be wrapper around the original function.
    #so it can execute code before and after it.
    
    def the_wrapper_around_the_original_function():
        
        #Put here the code you want to be executecd BEFORE the original fxn is called
        print("Before the function runs")
        
        #Call the function (using parentheses)
        a_function_to_decorate()
        
        #put here the code you want to be executed AFTER the original fxn is called
        print("After the function runs")
        
    #At this point, "a_function_to_decorate" Has never been executed.
    #We return the wrapper function we have just created.
    #The wrapper contains the function and the code to execute before and after. It's ready to use
    return the_wrapper_around_the_original_function

#Now imagine you create a function you don't want to touch ever.

def a_stand_alone_function():
    print("I am a stand alone function, don't you dare to modify me")

Now, you probably want that every time you call `a_stand_alone_function`, `a_stand_alone_function_decorated` is called instead. That's easy, just overwrite `a_stand_alone_function` with the function returned by `my_new_decorator`:

In [12]:
a_stand_alone_function = my_new_decorator(a_stand_alone_function)
a_stand_alone_function()

Before the function runs
I am a stand alone function, don't you dare to modify me
After the function runs


That's exactly what decorators do!

#### Decorators demystified

The previous example, using the decorator syntax:


In [14]:
@my_new_decorator
def another_stand_alone_function():
    print("Leave me alone")
    
another_stand_alone_function()    

Before the function runs
Leave me alone
After the function runs


Yes, that's all,it's that simple. `@decorator` is just a shortcut to:

    another_stand_alone_function = my_new_decorator(another_stand_alone_function)
  
Decorators are just a pythonic variant of the [decorator design pattern](). There are several classic design patterns embedded in Python to ease development (like iterators).

Of course, you can accumulate decorators:

In [15]:
def bread(func):
    def wrapper():
        print("</''''''\>")
        func()
        print("<\''''''/>")
    return wrapper

def ingredients(func):
    def wrapper():
        print("#tomatoes#")
        func()
        print("-salad-")
    return wrapper

def sandwich(food = "--ham--"):
    print(food)
    
sandwich()    

--ham--


In [16]:
sandwich =  bread(ingredients(sandwich))
sandwich()

</''''''\>
#tomatoes#
--ham--
-salad-
<''''''/>


Using the Python decorator syntax:

In [17]:
@bread
@ingredients
def sandwich(food="--ham--"):
    print(food)
    
sandwich()    

</''''''\>
#tomatoes#
--ham--
-salad-
<''''''/>


The order you set the decorators MATTERS:

In [18]:
@ingredients
@bread
def strange_sandwich(food = "--ham--"):
    print(food)
    
strange_sandwich()    

#tomatoes#
</''''''\>
--ham--
<''''''/>
-salad-


#### Now: to answer the question...

As a conclusion, you can easily see how to answer the question:

In [20]:
#The decorator to make it bold
def makebold(fn):
    #The new fxn the decorator returns
    def wrapper():
        #Insertion of some code before and after
        return "<b>"+fn()+"</b>"
    return wrapper

#The decorator to make it italic
def makeitalic(fn):
    def wrapper():
        return"<i>"+fn()+"</i>"
    return wrapper
@makebold
@makeitalic
def say():
    return "hello"

say()

'<b><i>hello</i></b>'

Above is equivalent to:

    say = makebold(makeitalic(say))


#### Taking decorators to the next level

##### Passing arguments to the decorated function

In [23]:
#It's not black magic, you just have to let the wrapper pass the argument:

def a_decorator_passing_args(func_to_decorate):
    def a_wrapper_accepting_args(arg1, arg2):
        print("I got args!Look:{0}, {1}".format(arg1,arg2))
        func_to_decorate(arg1, arg2)
    return a_wrapper_accepting_args

#since when you are calling the function returned by the decorator, you are calling the wrapper,
#passing arguments to the wrapper will let it pass them to the decorated function

@a_decorator_passing_args
def print_full_name(first, last):
    print("My name is {0} {1}".format(first, last))
    
print_full_name('Sam', 'Gates')    
    

I got args!Look:Sam, Gates
My name is Sam Gates


#### Decorating Methods

One nifty thing about Python is that methods and functions are really the same. The only difference is that methods expect that their first argument is a reference to the current object(`self`).

That means you can build a decorator for methods the same way! Just remember to take `self` into consideration:

In [25]:
def method_friendly_decorator(method_to_decorate):
    def wrapper(self, lie):
        lie = lie - 3 #vey friendly, decrease age even more :)
        return method_to_decorate(self, lie)
    return wrapper

class Lucy:
    def __init__(self):
        self.age = 32
        
    @method_friendly_decorator
    def sayYourAge(self,lie):
        print("I am {0}, what did you think?".format(self.age+lie))
        
l = Lucy()
l.sayYourAge(-3)

I am 26, what did you think?


If you're making general-purpose decorator--one you'll apply to any function or methods, no matter its arguments--then just use `*args, **kwargs`:

In [26]:
def a_decorator_passing_arbitrary_args(function_to_decorate):
    #the wrapper accepts any arguments
    def a_wrapper_accepting_arbitrary_args(*args,**kwargs):
        print("Do I have args?:")
        print(args)
        print(kwargs)
        #Then you unpack the arguments, here *args, **kwargs
        function_to_decorate(*args,**kwargs)
    return a_wrapper_accepting_arbitrary_args

@a_decorator_passing_arbitrary_args
def function_with_no_argument():
    print("Python is cool, no argument here.")
    
function_with_no_argument()    

Do I have args?:
()
{}
Python is cool, no argument here.


In [27]:
@a_decorator_passing_arbitrary_args
def function_with_arguments(a,b,c):
    print(a,b,c)
    
function_with_arguments(1,2,3) 

Do I have args?:
(1, 2, 3)
{}
1 2 3


In [29]:
@a_decorator_passing_arbitrary_args
def function_with_named_arguments(a,b,c, platypus = "why not?"):
    print("Do {0}, {1} and {2} like platypus? {3}".format(a,b,c,platypus))
    
function_with_named_arguments('Sam', 'Linus', 'Steve', platypus = 'Indeed!') 

Do I have args?:
('Sam', 'Linus', 'Steve')
{'platypus': 'Indeed!'}
Do Sam, Linus and Steve like platypus? Indeed!


In [31]:
class Mary:
    def __init__(self):
        self.age = 31
        
    @a_decorator_passing_arbitrary_args
    def sayYourAge(self, lie= -3):
        print("I am {0}".format(self.age+ lie))
        
m = Mary()

m.sayYourAge()

Do I have args?:
(<__main__.Mary object at 0x035F3DB0>,)
{}
I am 28


#### Passing arguments to the decorator

Great, now what would you say about passing arguments to the decorator itself?

This can get somewhat twisted, since a decorator must accept a function as an argument. Therefore you cannot pass the decorated function's arguments directly to the decorator. 

Before rushing to the solution, let's write a little reminder:

In [36]:
#decorators are ordinary functions\

def my_decorator(func):
    print("I am an ordinary function")
    def wrapper():
        print("I am function returned by the decorator")
        func()
    return wrapper

# therefore you can call it without any "@"

def lazy_function():
    print("zzzzzzz")
    
decorated_function = my_decorator(lazy_function)
# Take note that 'lazy_function' wasn't executed but 'print' function of decorator executed.


I am an ordinary function


In [37]:
@my_decorator
def lazy_function():
    print("zzzzzz")

I am an ordinary function


It's exactly the same. "my_decorator" is called. So when you `@my_decorator`, you are telling Python to call the function 'labelled by the variable '`my_decorator`'.

This is important! The label you give can point directly to the decorator -**or not**.

Let's get evil. :)

In [38]:
def decorator_maker():
    
    print("I make decorators! I am executed only once:"
         "when you make me create a decorator.")
    
    def my_decorator(func):
        print("I'm a decorator. I am executed only when you decorate a function.")
        
        def wrapped():
            print("I am the wrapper around the decorated function"
                 "I am called when you call the decorated function."
                 "As the wrapper, I return the result of the decorated function")
            return func()
        
        print("As the decorator, I returned the wrapped function")
        
        return wrapped
    
    print("As a decorator maker, I return a decorator")
    return my_decorator

#Let's create a decorator. It's just a new function after all

new_decorator = decorator_maker()

I make decorators! I am executed only once:when you make me create a decorator.
As a decorator maker, I return a decorator


In [39]:
#Now we decorate the function

def decorated_function():
    print("I am the decorated function.")
    
decorated_function = new_decorator(decorated_function)    

I'm a decorator. I am executed only when you decorate a function.
As the decorator, I returned the wrapped function


In [40]:
decorated_function()

I am the wrapper around the decorated functionI am called when you call the decorated function.As the wrapper, I return the result of the decorated function
I am the decorated function.


No surprise here.

Let's do Exactly the same thing, but skip all the pesky intermediate variables:

In [41]:
def decorated_function():
    print("I am the decorated function")
    
decorated_function =decorator_maker()(decorated_function)    

I make decorators! I am executed only once:when you make me create a decorator.
As a decorator maker, I return a decorator
I'm a decorator. I am executed only when you decorate a function.
As the decorator, I returned the wrapped function


In [42]:
decorated_function()


I am the wrapper around the decorated functionI am called when you call the decorated function.As the wrapper, I return the result of the decorated function
I am the decorated function


Let's make it even shorter

In [43]:
@decorator_maker()
def decorated_function():
    print("I am the decorated function")

I make decorators! I am executed only once:when you make me create a decorator.
As a decorator maker, I return a decorator
I'm a decorator. I am executed only when you decorate a function.
As the decorator, I returned the wrapped function


In [44]:
decorated_function()

I am the wrapper around the decorated functionI am called when you call the decorated function.As the wrapper, I return the result of the decorated function
I am the decorated function


Hey, did you see that? We used a function call with the `2` syntax! :)

So, back to decorators with arguments. If we can use functions to generate the decorator on the fly, we can pass arguments to that function, right?

In [46]:
def decorator_maker_with_args(dec_arg1, dec_arg2):
    print("I make decorators! And I accept arguments: {0}, {1}".format(dec_arg1, dec_arg2))
    
    def my_decorator(func):
        #The ability to pass arguments here is a gift from closures.
        #If you aren't comfortable with closures, you can assume it's ok, or read the SO link
        #link to be added later
        print("I am the decorator. Somehow you passed me args: {0},{1}".format(dec_arg1, dec_arg2))
        
        #Don't confuse decorator args with function args
        def wrapped(func_arg1,func_arg2):
            print("I am the wrapper around the decorated fxn. \n"
                 "I can access all the variables \n"
                 "\t - from the decorator: {0},{1}\n"
                 "\t - from the function call:{2},{3}\n"
                 "Then I can pass them to decorated function"
                 .format(dec_arg1, dec_arg2, func_arg1, func_arg2))
            return func(func_arg1, func_arg2)
            
        return wrapped
        
    return my_decorator

@decorator_maker_with_args("Leonard", "Sheldon")
def decorated_function_with_args(func_arg1, func_arg2):
    print("I am the decorated function and only knows abt my args: {0},{1}"
          .format(func_arg1, func_arg2))
    
decorated_function_with_args("Rajesh","Howard")    

I make decorators! And I accept arguments: Leonard, Sheldon
I am the decorator. Somehow you passed me args: Leonard,Sheldon
I am the wrapper around the decorated fxn. 
I can access all the variables 
	 - from the decorator: Leonard,Sheldon
	 - from the function call:Rajesh,Howard
Then I can pass them to decorated function
I am the decorated function and only knows abt my args: Rajesh,Howard


Here it is: a decorator with arguments. Arguments can be set as variable:

In [47]:
c1 = "Penny"
c2 = "Leslie"

@decorator_maker_with_args("Leonard", c1)
def decorated_function_with_args(func_arg1, func_arg2):
    print("I am decorated function and only knows about my args: {0}, {1}"
          .format(func_arg1, func_arg2))

decorated_function_with_args(c2, "Howard")    

I make decorators! And I accept arguments: Leonard, Penny
I am the decorator. Somehow you passed me args: Leonard,Penny
I am the wrapper around the decorated fxn. 
I can access all the variables 
	 - from the decorator: Leonard,Penny
	 - from the function call:Leslie,Howard
Then I can pass them to decorated function
I am decorated function and only knows about my args: Leslie, Howard


As you can see, you can pass arguments to the decorator like any function using the trick. You can even use `*args, **kwargs` if you wish. But remember decorators are called **only once**.

Just when Python imports the script. You can't dynamically set the arguments afterwards. When you do `import x`, **the function is already decorated**, so you can't change anything.

#### Let's practice: decorating a decorator

Okay, as a bonus, I'll give you a snipppet to make any decorator accept generically any argument. After all, in order to accept arguments, we created our decorator using another function.

We wrapped the decorator.

Anything else we saw recently that wrapped function?

Oh yes, decorators!

Let's have some fun and write a decorator for the decorators:

In [50]:
def decorator_with_args(decorator_to_enhance):
    """This function is supposed to be used as a decorator. It must decorate an other function,
    that is intended to be used as a decorator. Take a cup of coffee. It will allow any 
    decorator to accept any arbitray number of argumnts, saving you the headache to remember 
    how to that every time. """
    
    #we use the same trick we did to pass arguments
    def decorator_maker(*args, **kwargs):
        #we create on the fly a decorator that accept only a function
        #but keeps the passed arguments from the maker.
        def decorator_wrapper(func):
            #we return the result of the original decorator, which, after all, 
            #IS JUST AN ORDINARY FUNCTION which returns a function.
            #Only pitfall: the decorator must have this specific signature or it won't work
            return decorator_to_enhance(func, *args, **kwargs)
        return decorator_wrapper
    return decorator_maker

It can be used as follows:

In [52]:
@decorator_with_args
def decorated_decorator(func, *args, **kwargs):
    def wrapper(func_arg1, func_arg2):
        print("Decorate`d with {0} {1}".format(args, kwargs))
        return func(func_arg1, func_arg2)
    return wrapper

# Then you decorate the functions you wish with your brand new decorated decorator

@decorated_decorator(42,404, 1024)
def decorated_function(func_arg1, func_arg2):
    print("Hello {0} {1}".format(func_arg1, func_arg2))
    
decorated_function("Universe and ", "everything")    

Decorate`d with (42, 404, 1024) {}
Hello Universe and  everything


I know, the last time you had this feeling, it was after listening a guy saying: "before understanding recursion, you must first understand recursion". But now, don't you feel good about mastering this?

#### Best practices: decorators

 - Decorators were introduced in Python 2.4, so be sure your code will be run on >=2.4. 
 - Decorators slow down the function call. Keep that in mind. 
 - **You can't un-decorate a function**
 - Decorators wrap functions, which can make them hard to debug. (This gets better from Python >=2.5, see below)
 
The `functools` module was introduced in Python 2.5. It includes the function `functools.wraps()`, which copies the name, module, and docstring of the decorated function to its wrappe. (Fun fact: `functools.wraps()` is a decorator! 
  

In [53]:
#for debugging, the stacktrace prints you the function __name__

def foo():
    print("foo")
    
print(foo.__name__)    

foo


In [54]:
#with a decorator, it gets messy
def bar(func):
    def wrapper():
        print("bar")
        return func()
    return wrapper

@bar
def foo():
    print("foo")
    
print(foo.__name__)    

wrapper


In [55]:
#'functools' can help here

import functools

def bar(func):
    #we say that 'wrapper', is wrapping "func" and the magic begins
    @functools.wraps(func)
    def wrapper():
        print("bar")
        return func()
    return wrapper

@bar
def foo():
    print("foo")
    
print(foo.__name__)    

foo


#### How can the decorators be useful?

**Now the big question:** What can I use decorators for?

Seem cool and powerful, but a practical example would be great. Well, there are 1000 possibilities.Classic uses are extending a function behavior from an external lib(you can't modify it), or for debugging (you don't want to modify it because it's temporary.)

You can use them to extend several functions in a DRY's way, like so:

In [57]:
def benchmark(func):
    """A decorator that prints the time a function takes to execute"""
    import time
    def wrapper(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        print("{0} {1}".format(func.__name__, time.clock()-t))
        return res
    return wrapper

def logging(func):
    """A decorator that logs the activity of the script. It actually just print it, but could
    be logging!"""
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print("{0}, {1} {2}".format(func.__name__, args, kwargs))
        return res
    return wrapper

def counter(func):
    """A decorator that counts and prints the number of times a function has been run"""
    def wrapper(*args, **kwargs):
        wrapper.count = wrapper.count + 1
        res = func(*args, **kwargs)
        print("{0} has been used: {1}x".format(func.__name__, wrapper.count))
        return res
    wrapper.count = 0
    return wrapper

@counter
@benchmark
@logging
def reverse_string(string):
    return str(reversed(string))

print(reverse_string("Able was I ere I saw Elba"))
print(reverse_string("A man, a plan, a canoe, pasta, heros, rajahs"))

# reverse string isn't printed?           

reverse_string, ('Able was I ere I saw Elba',) {}
wrapper 0.0003452901720618256
wrapper has been used: 1x
reverse_string, ('A man, a plan, a canoe, pasta, heros, rajahs',) {}
wrapper 7.465733449985425e-05
wrapper has been used: 2x


Of course the good thing with decorators is that you can use them right away on almost anything without rewriting. DRY, I said:

In [64]:
@counter
@benchmark
@logging
def get_random_futurama_quote():
    from urllib import urlopen
    result = urlopen("http://subfusion.net/cgi-bin/quote.pl?quote=futurama").read()
    try:
        value = result.split("<br><b><hr><br>")[1].split("<br><br><hr>")[0]
        return value.strip()
    except:
        return "No, I'm .... doesn't!"
    
print(get_random_futurama_quote())
print(get_random_futurama_quote())
    

ImportError: cannot import name 'urlopen'

Python itself provides several decorators: `property`, `staticmethod` etc.

 - Django uses decorators to manage caching and view permissions.
 - Twisted to fake inlining asynchonous function calls.
 
This really is a large playground. 

In [65]:
import urllib

In [67]:
help(urllib)

Help on package urllib:

NAME
    urllib

PACKAGE CONTENTS
    error
    parse
    request
    response
    robotparser

FILE
    c:\miniconda3\envs\data\lib\urllib\__init__.py


